<a href="https://colab.research.google.com/github/devak23/python/blob/master/ml2/00_tensorflow_fundamentals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# In this notebook, we are going to cover some Tensor fundamentals.

More specifically, we are going to cover:
* Intro to tensors
* Getting information from tensors
* Manipulating tensors
* Tensors and Numpy
* Using @tf.function (a way to speed up your regular python functions)
* Using GPUs with tensorflow (or TPUs)
* Exercises

## Introduction to Tensors

In [ ]:
# Import Tensorflow
import tensorflow as tf
print(tf.__version__)

2.15.0


In [ ]:
# Creating the tensors with tf.constant()
scalar = tf.constant(7)
scalar

<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [ ]:
# Check the number of dimensions of a tensor found by ndim (ndim stands for number of dimensions)
scalar.ndim

0

In [ ]:
# Create a vector
vector = tf.constant([10, 10])
vector

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10, 10], dtype=int32)>

In [ ]:
# Check the dimension of vector
vector.ndim

1

In [ ]:
# Create a matrix
matrix = tf.constant([[10,7],
                      [7, 10]])
matrix

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 7, 10]], dtype=int32)>

In [ ]:
# Check the dimension of matrix
matrix.ndim

2

In [ ]:
# Create another matrix
another_matrix = tf.constant([[10., 7.],
                             [3., 2.,],
                             [8., 9.]], dtype=tf.float16) # specifies the data type with dtype parameter
another_matrix

<tf.Tensor: shape=(3, 2), dtype=float16, numpy=
array([[10.,  7.],
       [ 3.,  2.],
       [ 8.,  9.]], dtype=float16)>

In [ ]:
# check the dimension of another_matrix
another_matrix.ndim

2

In [ ]:
# Let's create a tensor
tensor = tf.constant([[[1, 2, 3],
                       [4, 5, 6]],
                      [[7, 8, 9],
                       [10, 11, 12]],
                      [[13, 14, 15],
                       [1, 17, 18]]])
tensor

<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[ 1,  2,  3],
        [ 4,  5,  6]],

       [[ 7,  8,  9],
        [10, 11, 12]],

       [[13, 14, 15],
        [ 1, 17, 18]]], dtype=int32)>

In [ ]:
# check the dimension of tensor
tensor.ndim

3

What we have created so far:
* scalar = single number
* vector = a number with direction (Ex: windspeed)
* Matrix = 2 dimensional array of numbers
* Tensor = n dimensional array of numbers (where n can be any number: 0 dimensional = scalar, 1 dimension = vector)

### Creating tensors with `tf.Variable`

In [ ]:
# Create the same tensor with tf.Variable as above
changeable_tensor = tf.Variable([10, 7])
unchangeable_tensor = tf.constant([10, 7])

changeable_tensor, unchangeable_tensor

(<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10,  7], dtype=int32)>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([10,  7], dtype=int32)>)

In [ ]:
# Lets try changing one of the elements in our changeable tensor
changeable_tensor[0] = 7

TypeError: 'ResourceVariable' object does not support item assignment

In [ ]:
# how about we try .assign()
changeable_tensor[0].assign(7)

<tf.Variable 'UnreadVariable' shape=(2,) dtype=int32, numpy=array([7, 7], dtype=int32)>

In [ ]:
# Lets try to assign some value to the unchangeable_tensor
unchangeable_tensor[0].assign(7)

AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'assign'

🔑 **Note:** Rarely in practise you will need to decide whether to use a`tf.constant` or `tf.Variable` to create tensors, as TensorFlow does this for you. However, if in doubt, use `tf.constant` and change it later if needed.

### Creating random Tensors

Random tensors are tensors of some arbitrary size which contain random numbers

In [ ]:
# create 2 random (but the same) tensors
random1_normal = tf.random.Generator.from_seed(42)
random1_normal = random1_normal.normal(shape=(3,2))
random2_normal = tf.random.Generator.from_seed(42)
random2_normal = random2_normal.normal(shape=(3,2))

# Are they equal
random1_normal, random2_normal, random1_normal == random2_normal

(<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193763, -1.8107855 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193763, -1.8107855 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=bool, numpy=
 array([[ True,  True],
        [ True,  True],
        [ True,  True]])>)

In [ ]:
# create 2 random (but the same) tensors
random_1 = tf.random.Generator.from_seed(42) # set seed for reproducibility
random_1 = random_1.normal(shape=(3,2))
random_2 = tf.random.Generator.from_seed(42)
random_2 = random_2.uniform(shape=(3,2))

# are they equal?
random_1, random_2, random_1 == random_2

(<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193763, -1.8107855 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[0.7493447 , 0.73561966],
        [0.45230794, 0.49039817],
        [0.1889317 , 0.52027524]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=bool, numpy=
 array([[False, False],
        [False, False],
        [False, False]])>)

### Shuffle the order of elements in a tensor

In [ ]:
# Shuffle a tensor (Valuable when you want to shuffle the data so that the inherent order of data doesn't affect learning)
not_shuffled = tf.constant([[10, 7],
                            [3, 4],
                            [2, 5]])
print (f"not_shuffled.ndim = {not_shuffled.ndim}")
not_shuffled

not_shuffled.ndim = 2


<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]], dtype=int32)>

In [ ]:
# shuffle our non-shuffled tensor
tf.random.set_seed(42) # set the global level random seed
tf.random.shuffle(not_shuffled, 42) # set operation level random seed

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]], dtype=int32)>

🏋 **Exercise:** Read through the Tensor flow documentation at: https://www.tensorflow.org/api_docs/python/tf/random/set_seed and practise writing 5 random tensors and shuffle them

It looks like if we want our shuffled tensors to be in the same order, we've got to use the global level and operational level random seed:
> Rule 4: "If both the global and the operation seed are set: Both seeds are used in conjunction to determine the random sequence."

**Note** that setting only the global seed doesn't help as it would still pick up an arbitrary operational seed.
> Rule 2: "If the global seed is set, but the operation seed is not: The system deterministically picks an operation seed in conjunction with the global seed so that it gets a unique random sequence. Within the same version of tensorflow and user code, this sequence is deterministic. However across different versions, this sequence might change. If the code depends on particular seeds to work, specify both global and operation-level seeds explicitly."

### Other ways to make tensors

In [ ]:
# Create a tensor of all ones
tf.ones([10, 7])

<tf.Tensor: shape=(10, 7), dtype=float32, numpy=
array([[1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.]], dtype=float32)>

In [ ]:
# Create a tensors of all zeros
tf.zeros(shape=(3, 4))

<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]], dtype=float32)>

### Turn numpy arrays into tensors

The main difference between numpy arrays and TensorFlow tensor is that tensors can be run on a GPU (much faster for numerical computing)

In [ ]:
# You can create numpy arrays into tensors
import numpy as np
numpy_A = np.arange(1, 25, dtype=np.int32) # create a numpy array between 1 and 25
numpy_A

# X = tf.constant(some_matrix) # capital for matrix/tensor
# y = tf.constant(vector) # non capital for vector

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24], dtype=int32)

In [ ]:
A = tf.constant(numpy_A, shape=(3, 4, 2))
B = tf.constant(numpy_A)
A, B


(<tf.Tensor: shape=(3, 4, 2), dtype=int32, numpy=
 array([[[ 1,  2],
         [ 3,  4],
         [ 5,  6],
         [ 7,  8]],
 
        [[ 9, 10],
         [11, 12],
         [13, 14],
         [15, 16]],
 
        [[17, 18],
         [19, 20],
         [21, 22],
         [23, 24]]], dtype=int32)>,
 <tf.Tensor: shape=(24,), dtype=int32, numpy=
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24], dtype=int32)>)

### Getting information from tensors

When dealing with tensors, you want to be aware of the following attributes:
* Shape
* Rank
* Axis or dimensions
* Size

In [ ]:
# Create a rank 4 tensor (4 dimensions)
rank_4_tensor = tf.zeros(shape=[2, 3, 4, 5])
rank_4_tensor

<tf.Tensor: shape=(2, 3, 4, 5), dtype=float32, numpy=
array([[[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]],


       [[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]]], dtype=float32)>

In [ ]:
rank_4_tensor.shape, rank_4_tensor.ndim, tf.size(rank_4_tensor)

(TensorShape([2, 3, 4, 5]), 4, <tf.Tensor: shape=(), dtype=int32, numpy=120>)

In [ ]:
2 * 3* 4 * 5

120

In [ ]:
# Get various attributes of our tensor
print("DataType of every element: ", rank_4_tensor.dtype)
print("Number of dimensions (rank): ", rank_4_tensor.ndim)
print("Shape of the tensor: ", rank_4_tensor.shape)
print("Elements along the 0 axis:", rank_4_tensor.shape[0])
print("Elements along the last axis:", rank_4_tensor.shape[-1])
print("Total number of elements in our tensor:", tf.size(rank_4_tensor).numpy())

DataType of every element:  <dtype: 'float32'>
Number of dimensions (rank):  4
Shape of the tensor:  (2, 3, 4, 5)
Elements along the 0 axis: 2
Elements along the last axis: 5
Total number of elements in our tensor: 120


### Indexing tensors

Tensors can be indexed just like python lists

In [ ]:
some_list = [1, 2, 3, 4]
some_list[:2]

[1, 2]

In [ ]:
# Get the first 2 elements of each dimension
rank_4_tensor[:2, :2, :2, :2]

<tf.Tensor: shape=(2, 2, 2, 2), dtype=float32, numpy=
array([[[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]]], dtype=float32)>

In [ ]:
some_list[:1]

[1]

In [ ]:
rank_4_tensor.shape

TensorShape([2, 3, 4, 5])

In [ ]:
# Get the first element from each dimension from each index except for the final one
rank_4_tensor[:1, :1, :1, :]

<tf.Tensor: shape=(1, 1, 1, 5), dtype=float32, numpy=array([[[[0., 0., 0., 0., 0.]]]], dtype=float32)>

In [ ]:
# Create a rank 2 tensor (2 dimension)
rank_2_tensor = tf.constant([[10, 7],
                             [3, 4]])
rank_2_tensor.shape, rank_2_tensor.ndim

(TensorShape([2, 2]), 2)

In [ ]:
# Get the last item of each of rank 2 tensor
rank_2_tensor[:, -1]

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([7, 4], dtype=int32)>

In [ ]:
# add in the extra dimension to our rank 2 tensor
rank_3_tensor = rank_2_tensor[...,tf.newaxis]
rank_3_tensor

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[10],
        [ 7]],

       [[ 3],
        [ 4]]], dtype=int32)>

In [ ]:
# Alternative to tf.newaxis
tf.expand_dims(rank_2_tensor, axis=-1) # -1 means expand the final axis

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[10],
        [ 7]],

       [[ 3],
        [ 4]]], dtype=int32)>

In [ ]:
# What happens if we change the axis=-1 to axis=0. When in doubt, try it out.
tf.expand_dims(rank_2_tensor, axis=0) # expand the 0- axis

<tf.Tensor: shape=(1, 2, 2), dtype=int32, numpy=
array([[[10,  7],
        [ 3,  4]]], dtype=int32)>

### Manipulating tensors (Tensor operations)

**Basic Operations**
`+`, `-`, `*`, `/`

In [ ]:
# You can add values to a tensor using the addition operator
tensor = tf.constant([[10, 7],
                      [3, 4]])
tensor + 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[20, 17],
       [13, 14]], dtype=int32)>

In [ ]:
# Original tensor is unchanged
tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4]], dtype=int32)>

In [ ]:
tensor - 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 0, -3],
       [-7, -6]], dtype=int32)>

In [ ]:
tensor * 3

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[30, 21],
       [ 9, 12]], dtype=int32)>

In [ ]:
# We can use the tensorflow builtin functions too
tf.divide(tensor,2)

<tf.Tensor: shape=(2, 2), dtype=float64, numpy=
array([[5. , 3.5],
       [1.5, 2. ]])>

**Matrix Multiplication**

In machine learning , matrix multiplication is one of the most common tensor operations. The operations we did so far is 'element-wise'

There are two rules our tensors (or matrices) need to fulfill if we are going to matrix multiply them:

1. The inner dimension must match
2. The resulting matrix has the shape of outer dimension

In [ ]:
# Matrix multiplication in tensor flow
print(tensor)
tf.matmul(tensor, tensor)

tf.Tensor(
[[10  7]
 [ 3  4]], shape=(2, 2), dtype=int32)


<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[121,  98],
       [ 42,  37]], dtype=int32)>

In [ ]:
# However note that tensor * tensor yields a different result
tensor * tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[100,  49],
       [  9,  16]], dtype=int32)>

In [ ]:
tf.matmul(t1, t2)

NameError: name 't1' is not defined

In [ ]:
# Matrix multiplication using the "@". This has the same effect as tf.matmul that we saw earlier.
tensor @ tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[121,  98],
       [ 42,  37]], dtype=int32)>

In [ ]:
tensor.shape

TensorShape([2, 2])

In [ ]:
# Lets multiply 2 different tensors this time
t1 = tf.constant([1, 2, 5, 7, 2, 1, 3, 3, 3], shape=(3, 3))
t2 = tf.constant([3, 5, 6, 7, 1, 8], shape=(3,2))
t1, t2

(<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
 array([[1, 2, 5],
        [7, 2, 1],
        [3, 3, 3]], dtype=int32)>,
 <tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[3, 5],
        [6, 7],
        [1, 8]], dtype=int32)>)

In [ ]:
t3 = tf.constant([[1, 2],
                  [3, 4],
                  [5, 6]])
t4 = tf.constant([[7, 8],
                  [9, 10],
                  [11, 12]])
t3, t4

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [5, 6]], dtype=int32)>,
 <tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[ 7,  8],
        [ 9, 10],
        [11, 12]], dtype=int32)>)

In [ ]:
t3 @ t4

InvalidArgumentError: {{function_node __wrapped__MatMul_device_/job:localhost/replica:0/task:0/device:CPU:0}} Matrix size-incompatible: In[0]: [3,2], In[1]: [3,2] [Op:MatMul] name: 

In [ ]:
# if we now change t4 such as to create a completely new tensor...
t5 = tf.constant([[7, 8, 9],
                  [10, 11, 12]])
t3, t5

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [5, 6]], dtype=int32)>,
 <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[ 7,  8,  9],
        [10, 11, 12]], dtype=int32)>)

In [ ]:
# The inner dimensions of t3 i.e shape (3,2) matches t5 shape (2, 3).
# Thinking in terms of rows and columns t3 has 2 columns and t5 has 2 rows and
# hence matrix multiplication should work now.
t3 @ t5

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]], dtype=int32)>

In [ ]:
# Alternatively instead of creating a totally new tensor t5, we could "reshape" t4 itself
tf.reshape(t4, shape=(2, 3))

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[ 7,  8,  9],
       [10, 11, 12]], dtype=int32)>

In [ ]:
t3.shape, tf.reshape(t4, shape=(2,3)).shape

(TensorShape([3, 2]), TensorShape([2, 3]))

In [ ]:
# So you see in the above code the inner dimensions (2 and 2 match). Now lets try to multiply
t3 @ tf.reshape(t4, shape=(2, 3))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]], dtype=int32)>

In [ ]:
# If you try to work the other way around, that should work too. That means
# reshaping t3 to match the inner dimension of t4. So lets reshape t3 to have
# 2 "rows" and 3 "columns" (thinking in terms of rows and columns just for ease)
tf.reshape(t3, shape=(2, 3)) @ t4

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 58,  64],
       [139, 154]], dtype=int32)>

and that worked too. The output will be different as in the above case we get a 2x2 matrix and in the previous case we got a 3x3 matrix. That sits well with our second rule as described in the rules section.

In [ ]:
# Can do the same with transpose
t3, tf.transpose(t3), tf.reshape(t3, shape=(2, 3))

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [5, 6]], dtype=int32)>,
 <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[1, 3, 5],
        [2, 4, 6]], dtype=int32)>,
 <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[1, 2, 3],
        [4, 5, 6]], dtype=int32)>)

**IMPORTANT:** So the difference between reshape and transpose is that transpose shifts the axis whereas reshape just shuffles the matrix into the shape you want.

In [ ]:
# try matrix multiplication with transpose rather than matmul
tf.matmul(tf.transpose(t3), t4)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 89,  98],
       [116, 128]], dtype=int32)>

**The dot product**

Matrix multiplication is also referred to as the dot product.

You can perform matrix multiplication using:

* `tf.matmul()`
* `tf.tensordot()`


In [ ]:
# Perform the dot product on X and Y (requires X or Y to be transposed)
X = t3
Y = t4
X, Y

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [5, 6]], dtype=int32)>,
 <tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[ 7,  8],
        [ 9, 10],
        [11, 12]], dtype=int32)>)

In [ ]:
tf.tensordot(tf.transpose(X), Y, axes=1)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 89,  98],
       [116, 128]], dtype=int32)>

In [ ]:
# Perform matrix multiplication between X and Y (transposed)
tf.matmul(X, tf.transpose(Y))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 23,  29,  35],
       [ 53,  67,  81],
       [ 83, 105, 127]], dtype=int32)>

In [ ]:
# Perform matrix multiplication between X and Y (reshaped)
tf.matmul(X, tf.reshape(Y, shape=(2, 3)))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]], dtype=int32)>

In [ ]:
# Check the value of Y, reshape Y and transposed Y
print("Normal Y:", Y, "\n")
print("Y reshaped to (2, 3): ", tf.reshape(Y, shape=(2,3)), "\n")
print("Y transposed: ", tf.transpose(Y))

Normal Y: tf.Tensor(
[[ 7  8]
 [ 9 10]
 [11 12]], shape=(3, 2), dtype=int32) 

Y reshaped to (2, 3):  tf.Tensor(
[[ 7  8  9]
 [10 11 12]], shape=(2, 3), dtype=int32) 

Y transposed:  tf.Tensor(
[[ 7  9 11]
 [ 8 10 12]], shape=(2, 3), dtype=int32)


Generally when performing multiplication on two tensors and one of the axes doesn't line up, you will tranpose rather than reshape one of the tensor to satisfy matrix multiplication rules

### Changing the datatype of a tensor

In [ ]:
# Create a new tensor with default datatype (float 32)
B = tf.constant([1.7, 7.4])
B, B.dtype

(<tf.Tensor: shape=(2,), dtype=float32, numpy=array([1.7, 7.4], dtype=float32)>,
 tf.float32)

In [ ]:
C = tf.constant([7, 10])
C, C.dtype

(<tf.Tensor: shape=(2,), dtype=int32, numpy=array([ 7, 10], dtype=int32)>,
 tf.int32)

In [ ]:
# change from float32 to float 16 (reduced precision)
D = tf.cast(B, dtype=tf.float16)
D, D.dtype

(<tf.Tensor: shape=(2,), dtype=float16, numpy=array([1.7, 7.4], dtype=float16)>,
 tf.float16)

In [ ]:
# Change from int32 to float32
E = tf.cast(C, dtype=tf.float32)
E

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([ 7., 10.], dtype=float32)>

In [ ]:
E_float16 = tf.cast(E, dtype=tf.float16)
E_float16

<tf.Tensor: shape=(2,), dtype=float16, numpy=array([ 7., 10.], dtype=float16)>

### Aggregating  tensors

Aggregating tensors = condensing them from multiple values down to a smaller amount of values


In [ ]:
# Getting the absolute values
D = tf.constant([-7, -10])
D

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([ -7, -10], dtype=int32)>

In [ ]:
tf.abs(D)

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([ 7, 10], dtype=int32)>

Let's go through the following forms of aggregation
* Get minimum
* Get maximum
* Get mean of a tensor
* Get the sum of a tensor

In [ ]:
# Create a random tensor with values between 0 and 100 of size 50
E = tf.constant(np.random.randint(0, 100, size=50))
E

<tf.Tensor: shape=(50,), dtype=int64, numpy=
array([97, 48,  5, 39, 33, 13, 38, 47,  9, 22, 80, 46, 39, 84, 12, 48, 91,
       80, 69,  7, 74, 92, 27,  6, 12, 38, 67, 79, 77, 83, 54, 78, 45, 78,
       90,  8, 16, 97, 72, 97, 18, 38, 85, 14, 93, 39,  3, 55, 99,  1])>

In [ ]:
tf.size(E), E.shape, E.ndim

(<tf.Tensor: shape=(), dtype=int32, numpy=50>, TensorShape([50]), 1)

In [ ]:
# Find the minimum
tf.reduce_min(E), tf.reduce_min(E).numpy(), np.min(E)

(<tf.Tensor: shape=(), dtype=int64, numpy=1>, 1, 1)

In [ ]:
# Find the maximum
tf.reduce_max(E), tf.reduce_max(E).numpy(), np.max(E)

(<tf.Tensor: shape=(), dtype=int64, numpy=99>, 99, 99)

In [ ]:
# Find the mean
tf.reduce_mean(E), tf.reduce_mean(E).numpy(), np.mean(E)

(<tf.Tensor: shape=(), dtype=int64, numpy=50>, 50, 50.84)

In [ ]:
# Find the sum
tf.reduce_sum(E), tf.reduce_sum(E).numpy(), np.sum(E)

(<tf.Tensor: shape=(), dtype=int64, numpy=2542>, 2542, 2542)

🏋**Exercise:** With what we have learnt so far, find the variance and standard deviation of our `E` tensor using tensorflow methods

In [ ]:
# Find variance
tf.reduce_variance(E) # this doesn't work

AttributeError: module 'tensorflow' has no attribute 'reduce_variance'

In [ ]:
# To find the variance of our tensor, we need access to the probability module in tensorflow
import tensorflow_probability as tfp
tfp.stats.variance(E), tfp.stats.variance(E).numpy(), np.var(E)

(<tf.Tensor: shape=(), dtype=int64, numpy=1002>, 1002, 1001.9744000000001)

In [ ]:
# the same thing can be achieved from the math package
tf.math.reduce_variance(tf.cast(E, dtype=tf.float32)), tf.math.reduce_variance(tf.cast(E, dtype=tf.float32)).numpy(), np.var(E)

(<tf.Tensor: shape=(), dtype=float32, numpy=1001.97437>,
 1001.97437,
 1001.9744000000001)

In [ ]:
# Find the standard deviation
tfp.stats.stddev(tf.cast(E, dtype=tf.bfloat16)), tfp.stats.stddev(tf.cast(E, dtype=tf.bfloat16)).numpy(), np.std(E)

(<tf.Tensor: shape=(), dtype=bfloat16, numpy=31.625>,
 31.625,
 31.653979212730903)

In [ ]:
# You can also use the standard deviation from the math package
tf.math.reduce_std(tf.cast(E, dtype=tf.float32)), tf.math.reduce_std(tf.cast(E, dtype=tf.float32)).numpy(), np.std(E)

(<tf.Tensor: shape=(), dtype=float32, numpy=31.653978>,
 31.653978,
 31.653979212730903)

### Find the positional maximum and minimum of a tensor



In [ ]:
# Create a new tensor for finding the positional max and min
tf.random.set_seed(42)
F = tf.random.uniform(shape=[50])
F

<tf.Tensor: shape=(50,), dtype=float32, numpy=
array([0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
       0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
       0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
       0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
       0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
       0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
       0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
       0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
       0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
       0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043],
      dtype=float32)>

In [ ]:
# Find the positional maximum
tf.argmax(F)

<tf.Tensor: shape=(), dtype=int64, numpy=42>

In [ ]:
# Index on our largest value
F[tf.argmax(F)]

<tf.Tensor: shape=(), dtype=float32, numpy=0.9671384>

In [ ]:
# Find the max value of F
tf.reduce_max(F)

<tf.Tensor: shape=(), dtype=float32, numpy=0.9671384>

In [ ]:
# Check for quality
F[tf.argmax(F)] == tf.reduce_max(F)

<tf.Tensor: shape=(), dtype=bool, numpy=True>

In [ ]:
# find the positional minimum
tf.argmin(F)

<tf.Tensor: shape=(), dtype=int64, numpy=16>

In [ ]:
# Find the min value of F
F[tf.argmin(F)], tf.reduce_min(F)

(<tf.Tensor: shape=(), dtype=float32, numpy=0.009463668>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.009463668>)

### Squeezing a tensor (removing all the single dimensions)

In [ ]:
# Create a tensor to get started
tf.random.set_seed(42)
G = tf.constant(tf.random.uniform(shape=[50]), shape=(1, 1, 1, 1, 50))
G

<tf.Tensor: shape=(1, 1, 1, 1, 50), dtype=float32, numpy=
array([[[[[0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
           0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
           0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
           0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
           0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
           0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
           0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
           0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
           0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
           0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043]]]]],
      dtype=float32)>

In [ ]:
G.shape

TensorShape([1, 1, 1, 1, 50])

In [ ]:
G_squeezed = tf.squeeze(G)
G_squeezed, G.shape, G_squeezed.shape

(<tf.Tensor: shape=(50,), dtype=float32, numpy=
 array([0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
        0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
        0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
        0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
        0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
        0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
        0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
        0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
        0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
        0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043],
       dtype=float32)>,
 TensorShape([1, 1, 1, 1, 50]),
 TensorShape([50]))

### One-Hot encoding in tensor

In [ ]:
# Create a list of indices
some_list = [0, 1, 2, 3] # could be red, green, blue, purple

# one-hot encode our list of indices
one_hot_encoded = tf.one_hot(some_list, depth=4)
one_hot_encoded

<tf.Tensor: shape=(4, 4), dtype=float32, numpy=
array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]], dtype=float32)>

In [ ]:
# Specify custom values for one hot encoding
tf.one_hot(some_list, depth=4, on_value="Present", off_value="Absent")

<tf.Tensor: shape=(4, 4), dtype=string, numpy=
array([[b'Present', b'Absent', b'Absent', b'Absent'],
       [b'Absent', b'Present', b'Absent', b'Absent'],
       [b'Absent', b'Absent', b'Present', b'Absent'],
       [b'Absent', b'Absent', b'Absent', b'Present']], dtype=object)>

### Squaring, log, square root

In [ ]:
# Create a new tensor
H = tf.range(1, 10)
H

<tf.Tensor: shape=(9,), dtype=int32, numpy=array([1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>

In [ ]:
# Square it
H_square = tf.square(H)
H_square

<tf.Tensor: shape=(9,), dtype=int32, numpy=array([ 1,  4,  9, 16, 25, 36, 49, 64, 81], dtype=int32)>

In [ ]:
# Log
tf.math.log(tf.cast(H, dtype=tf.float16))

<tf.Tensor: shape=(9,), dtype=float16, numpy=
array([0.    , 0.6934, 1.099 , 1.387 , 1.609 , 1.792 , 1.946 , 2.08  ,
       2.197 ], dtype=float16)>

In [ ]:
# Find the square root
H_square_sqrt = tf.sqrt(tf.cast(H_square, dtype=tf.float16))
H_square_sqrt, tf.cast(H_square_sqrt, dtype=tf.int16)

(<tf.Tensor: shape=(9,), dtype=float16, numpy=array([1., 2., 3., 4., 5., 6., 7., 8., 9.], dtype=float16)>,
 <tf.Tensor: shape=(9,), dtype=int16, numpy=array([1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int16)>)

### Tensors and Numpy

Tensorflow interacts beautifully with numpy arrays

🔑 **Note:** One of the main differences between a tensorflow and numpy array is that tensorflow tensor can be run on a GPU or a TPU (for fast numerical processing)

In [ ]:
# Create a tensor directly from numpy array
J = tf.constant(np.array([3., 7. ,10.]))
J

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 3.,  7., 10.])>

In [ ]:
# Convert our tensor to a numpy array
np.array(J), type(np.array(J))

(array([ 3.,  7., 10.]), numpy.ndarray)

In [ ]:
# Convert tensor J to numpy array
J.numpy(), type(J.numpy())

(array([ 3.,  7., 10.]), numpy.ndarray)

In [ ]:
J = tf.constant([3.])
J.numpy()[0]

3.0

In [ ]:
# Default types of each are slightly different
np_J = tf.constant(np.array([3. ,1.]))
numpy_J = tf.constant([3., 7., 10.])
tensor_J = tf.constant([3., 7., 10.])
np_J.dtype, numpy_J.dtype, tensor_J.dtype

(tf.float64, tf.float32, tf.float32)

### Finding access to GPU's

In [ ]:
import tensorflow as tf
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [ ]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
!nvidia-smi

Fri Jan 12 11:56:43 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P0              30W /  70W |    103MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

> 🔑 **NOTE:** If you have access to CUDA enabled GPU, TensorFlow will automatically use it whenever possible